In [1]:
taxi = pd.read_csv("/Users/jisuim/Desktop/tp_taxi/jisu/train.csv")

In [2]:
from sklearn.tree import DecisionTreeRegressor

In [3]:
taxi_new = taxi.copy()

# Feature Engineering

In [4]:
from math import cos, asin, sqrt
def dist(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295     #Pi/180
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a))

In [5]:
for i in range(len(taxi)):
    lat1 = taxi['pickup_latitude'][i]
    lon1 = taxi['pickup_longitude'][i]
    lat2 = taxi['dropoff_latitude'][i]
    lon2 = taxi['dropoff_longitude'][i]
    taxi_new.at[i, 'distance'] = dist(lat1, lon1, lat2, lon2)


In [7]:
Morning = ['01','02','03','04','05','06','07']
Night = ['19', '20', '21', '22', '23', '00']
for i in range(len(taxi_new)):
    k = taxi['pickup_datetime'][i][11:13]
    if k in Morning:
        taxi_new.at[i, "time_n"] = "Morning"
    elif k in Night:
        taxi_new.at[i, "time_n"] = "Night"
    else:
        taxi_new.at[i, "time_n"] = "Duty"
        
     

In [11]:
taxi_new.loc[taxi_new.distance > 200] = np.nan ##200km 넘는 데이터 제거
taxi_new.loc[taxi_new.trip_duration > 30000] = np.nan ##40000초(약 11시간)가 넘는 데이터 제거
taxi_new.loc[taxi_new.passenger_count == 0] = np.NAN   ### passenger 수가 0인 데이터 제거
taxi_new.dropna(inplace=True)

In [ ]:
dummy_time_n = pd.get_dummies(taxi_new['time_n'],prefix='time_n')
dummy_vendor_id = pd.get_dummies(taxi_new['vendor_id'], prefix='id')
dummy_store = pd.get_dummies(taxi_new['store_and_fwd_flag'], prefix='snf')

In [12]:
len(taxi_new)

1456542

In [13]:
len(taxi)

1458644

In [14]:
taxi_new.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,distance,time_n
0,id2875421,2.0,2016-03-14 17:24:55,2016-03-14 17:32:30,1.0,-73.982155,40.767937,-73.964630,40.765602,N,455.0,1.498521,Duty
1,id2377394,1.0,2016-06-12 00:43:35,2016-06-12 00:54:38,1.0,-73.980415,40.738564,-73.999481,40.731152,N,663.0,1.805507,Night
2,id3858529,2.0,2016-01-19 11:35:24,2016-01-19 12:10:48,1.0,-73.979027,40.763939,-74.005333,40.710087,N,2124.0,6.385098,Duty
3,id3504673,2.0,2016-04-06 19:32:31,2016-04-06 19:39:40,1.0,-74.010040,40.719971,-74.012268,40.706718,N,429.0,1.485498,Night
4,id2181028,2.0,2016-03-26 13:30:55,2016-03-26 13:38:10,1.0,-73.973053,40.793209,-73.972923,40.782520,N,435.0,1.188588,Duty


> passenger_count, 출발위치, 도착위치, 시간대, 직선거리, vendor_id, store_and_fwd_flag

# Decision Tree

In [19]:
X = taxi_new[['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude', 'distance']]

In [20]:
X.head()

,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,distance
0,40.767937,-73.982155,40.765602,-73.964630,1.498521
1,40.738564,-73.980415,40.731152,-73.999481,1.805507
2,40.763939,-73.979027,40.710087,-74.005333,6.385098
3,40.719971,-74.010040,40.706718,-74.012268,1.485498
4,40.793209,-73.973053,40.782520,-73.972923,1.188588


In [21]:
dfX = pd.concat([X, dummy_store, dummy_time_n, dummy_vendor_id], axis=1)

In [22]:
dfX.head()

,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,distance,snf_N,snf_Y,time_n_Duty,time_n_Morning,time_n_Night,id_1.0,id_2.0
0,40.767937,-73.982155,40.765602,-73.964630,1.498521,1,0,1,0,0,0,1
1,40.738564,-73.980415,40.731152,-73.999481,1.805507,1,0,0,0,1,1,0
2,40.763939,-73.979027,40.710087,-74.005333,6.385098,1,0,1,0,0,0,1
3,40.719971,-74.010040,40.706718,-74.012268,1.485498,1,0,0,0,1,0,1
4,40.793209,-73.973053,40.782520,-73.972923,1.188588,1,0,1,0,0,0,1


In [23]:
y = taxi_new['trip_duration']

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dfX, y, test_size=0.25)

In [31]:
tree = DecisionTreeRegressor(max_depth=7, random_state=0)
tree.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=7, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')

In [35]:
tree.score(X_train, y_train)

0.665690991735253

In [36]:
tree.score(X_test, y_test)

0.6666174034048411

In [34]:
tree.feature_importances_

array([4.61023869e-04, 4.77581572e-03, 2.16416852e-02, 5.09955853e-03,
       9.01013369e-01, 0.00000000e+00, 0.00000000e+00, 6.10044740e-02,
       3.63517112e-03, 2.27297661e-03, 0.00000000e+00, 9.59258714e-05])

# test.csv

In [37]:
taxi_test = pd.read_csv("/Users/jisuim/Desktop/tp_taxi/jisu/test.csv")

In [38]:
for i in range(len(taxi_test)):
    lat1 = taxi_test['pickup_latitude'][i]
    lon1 = taxi_test['pickup_longitude'][i]
    lat2 = taxi_test['dropoff_latitude'][i]
    lon2 = taxi_test['dropoff_longitude'][i]
    taxi_test.at[i, 'distance'] = dist(lat1, lon1, lat2, lon2)


In [39]:
### 시간대 19~00(Night) / 01~07(Morning) / 08~18(Duty) 로 분할하기
Morning = ['01','02','03','04','05','06','07']
Night = ['19', '20', '21', '22', '23', '00']
for i in range(len(taxi_test)):
    k = taxi_test['pickup_datetime'][i][11:13]
    if k in Morning:
        taxi_test.at[i, "time_n"] = "Morning"
    elif k in Night:
        taxi_test.at[i, "time_n"] = "Night"
    else:
        taxi_test.at[i, "time_n"] = "Duty"
        
     

In [40]:
tdummy_time_n = pd.get_dummies(taxi_test['time_n'],prefix='time_n')
tdummy_vendor_id = pd.get_dummies(taxi_test['vendor_id'], prefix='id')
tdummy_store = pd.get_dummies(taxi_test['store_and_fwd_flag'], prefix='snf')

In [41]:
X_t = taxi_test[['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude', 'distance']]

In [42]:
dfX_t = pd.concat([X_t, tdummy_store, tdummy_time_n, tdummy_vendor_id], axis=1)

In [43]:
t_y = tree.predict(dfX_t)

In [47]:
t_y = pd.DataFrame(t_y)

In [48]:
t_y.head()

,0
0,782.666574
1,782.666574
2,470.893991
3,1189.425024
4,388.223893


In [49]:
submission_test = pd.concat([taxi_test['id'], t_y], axis=1)

In [50]:
submission_test.to_csv('submission_2.csv', sep=',', na_rep='NaN')